In [7]:
from policyengine import PolicyEngineUK
from IPython.display import Markdown

uk = PolicyEngineUK()

uk_baseline, _ = uk.create_microsimulations(dict())

uk_poverty_rate = uk_baseline.calc("in_poverty_bhc", map_to="person").mean()
uk_gini = uk_baseline.calc("equiv_household_net_income", map_to="person").gini()

ni_baseline, _ = uk.create_microsimulations(dict(baseline_country_specific="NORTHERN_IRELAND"))

ni_poverty_rate = ni_baseline.calc("in_poverty_bhc", map_to="person").mean()
ni_gini = ni_baseline.calc("equiv_household_net_income", map_to="person").gini()

Markdown(
    f"""
# Northern Ireland UBI Feasibility Study

## Executive summary

Following proposals by the UBI Lab Northern Ireland, we analyse a range of UBI policies for Northern Ireland, funded by income tax reforms. For our analyses, we use [PolicyEngine UK](https://policyengine.org.uk), an app for designing and computing impacts of customisable tax and benefit reforms.

We find that...

## Northern Ireland today

Northern Ireland’s demographic and economic characteristics differ from those of the UK and other UK nations. In the main, this is lower economic output, but consistent levels of social security spending. NI has a GDP per capita that is around 20% lower than the UK average,[^1] and the population of 1.9m individuals generates around £2.8bn in Income Tax liabilities.[^2] Higher levels of public spending (with a net fiscal deficit of £9,500 per capita)[^3] in NI increase disposable incomes to offset this. As a result, we estimate that Northern Ireland's poverty rate is {ni_poverty_rate:.1%}, compared to the UK rate of {uk_poverty_rate:.1%}. We also find that it is more equal, with a Gini index of {ni_gini:.3f} against the UK's {uk_gini:.3f}. 

Many of Northern Ireland’s taxes and benefits operate under centralised UK rules, though exceptions to this include local property taxes such as domestic rates (and rates rebates), which exist in place of Council Tax and Council Tax Benefit. Unlike Scotland, Northern Ireland does not set alternative tax rates for Income Tax.

[^1]: [Northern Ireland Statistics Research Agency, 2020.](https://www.nisra.gov.uk/statistics/economic-output-statistics/gross-value-added-and-gross-domestic-product#:~:text=In%20terms%20of%20GDP%20per,78.6%20per%20cent%20of%20UK)

[^2]: [HMRC, 2019.](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/853118/Disaggregated_tax_and_NICs_receipts_-_methodological_note.pdf)

[^3]: [ONS, 2021](https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicsectorfinance/articles/countryandregionalpublicsectorfinances/financialyearending2021#:~:text=On%20a%20per%20person%20basis,per%20head%20was%20%C2%A34%2C74)

[^4]: Absolute poverty rate before housing costs, using the OpenFisca UK calibrated Family Resources Survey extrapolated to 2022.

"""
)


# Northern Ireland UBI Feasibility Study

## Executive summary

Following proposals by the UBI Lab Northern Ireland, we analyse a range of UBI policies for Northern Ireland, funded by income tax reforms. For our analyses, we use [PolicyEngine UK](https://policyengine.org.uk), an app for designing and computing impacts of customisable tax and benefit reforms.

We find that...

## Northern Ireland today

Northern Ireland’s demographic and economic characteristics differ from those of the UK and other UK nations. In the main, this is lower economic output, but consistent levels of social security spending. NI has a GDP per capita that is around 20% lower than the UK average,[^1] and the population of 1.9m individuals generates around £2.8bn in Income Tax liabilities.[^2] Higher levels of public spending (with a net fiscal deficit of £9,500 per capita)[^3] in NI increase disposable incomes to offset this. As a result, we estimate that Northern Ireland's poverty rate is 6.1%, compared to the UK rate of 5.5%. We also find that it is more equal, with a Gini index of 0.274 against the UK's 0.333. 

Many of Northern Ireland’s taxes and benefits operate under centralised UK rules, though exceptions to this include local property taxes such as domestic rates (and rates rebates), which exist in place of Council Tax and Council Tax Benefit. Unlike Scotland, Northern Ireland does not set alternative tax rates for Income Tax.

[^1]: [Northern Ireland Statistics Research Agency, 2020.](https://www.nisra.gov.uk/statistics/economic-output-statistics/gross-value-added-and-gross-domestic-product#:~:text=In%20terms%20of%20GDP%20per,78.6%20per%20cent%20of%20UK)

[^2]: [HMRC, 2019.](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/853118/Disaggregated_tax_and_NICs_receipts_-_methodological_note.pdf)

[^3]: [ONS, 2021](https://www.ons.gov.uk/economy/governmentpublicsectorandtaxes/publicsectorfinance/articles/countryandregionalpublicsectorfinances/financialyearending2021#:~:text=On%20a%20per%20person%20basis,per%20head%20was%20%C2%A34%2C74)

[^4]: Absolute poverty rate before housing costs, using the OpenFisca UK calibrated Family Resources Survey extrapolated to 2022.



Many of Northern Ireland’s taxes and benefits operate under centralised UK rules, though exceptions to this include local property taxes such as domestic rates (and rates rebates), which exist in place of Council Tax and Council Tax Benefit. Unlike Scotland, Northern Ireland does not set alternative tax rates for Income Tax.

A 2017 poll by Ipsos MORI found that 66% of Northern Irish people favoured a basic income, compared to 49% of UK residents.[^5] Northern Ireland's net favourability of +55 was more than double that of any other region surveyed.

[^5]: This is the only poll to ask about basic income for Northern Ireland and the UK, per the [UBI Center's Poll Tracker](http://polls.ubicenter.org).

## Exploring the space of feasible policies

One could model an infinite number of basic income policies for Northern Ireland. We constrained the problem by defining UBI parameter values, tax variables to adjust, and requirements for the resulting policy.

In particular, we limited to UBI policies of the following form:
* Amounts per adult (aged over 18) of £200, £300, and £400 per month
* Amounts per child of half the adult amount
* UBI payments do not count as income for tax or means-tested benefits

To fund the UBI, we considered three levers:
* Personal allowance, from zero to the current value of £12,570 (we also adjusted the higher rate threshold to keep it at £50,270)
* Basic rate increase
* Ratio of higher and additional rate increases to the basic rate increase, of 0x (not changing the higher and additional rates), 1x, and 2x

Finally, we constrained the end policy to:
* Be budget-neutral (assuming no behavioural responses)
* Avoid raising marginal tax rates above 100%,[^6] either via the normal tax rates, the phase-out of the personal allowance or benefit phase-outs

Figure 2 represents this exploratory exercise: each point in these lines is a feasible policy. For example, the leftmost line in each group shows the set of all policies that provide £200 per month to adults (£100 to children), and in which the higher and additional rates rise two percentage points per percentage point increase in the basic rate. It varies by the basic rate and personal allowance. The top-left point in this line is a policy that abolishes the Personal Allowance, increases the basic rate by 4p, and increases the higher and additional rates by 8p (see it in PolicyEngine [here]()).

[^6]: We exclude the Child Benefit High-Income Tax Charge from contributing to this limit, because the MTR addition from that is dependent on the number of children, with no upper bound.

## Costing the UBI

* Cost of UBI up to £400 per month
* Start with amount from personal allowance
* Show equivalence between each rate increase


In [1]:
from policyengine import PolicyEngineUK
from openfisca_uk.model_api import *
from openfisca_uk.api import *
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import bisect

uk = PolicyEngineUK()

_, baseline = uk.create_microsimulations(dict(baseline_country_specific="NORTHERN_IRELAND"))

def get_ubi_cost(
    adult_amount: float = 400 * 12,
    child_amount: float = 200 * 12,
    senior_amount: float = 100 * 12,
):
    in_ni = baseline.calc("country", map_to="person") == "NORTHERN_IRELAND"
    num_adults = baseline.calc("is_WA_adult")[in_ni].sum()
    num_children = baseline.calc("is_child")[in_ni].sum()
    num_seniors = baseline.calc("is_SP_age")[in_ni].sum()
    return adult_amount * num_adults + child_amount * num_children + senior_amount * num_seniors



In [2]:
# Silence warnings
import warnings
warnings.filterwarnings("ignore")

def ubi_ni_reform(
    personal_allowance: float = 12_570,
    basic_rate_addition: float = 0,
    higher_rate_addition: float = 0,
    additional_rate_addition: float = 0,
) -> Reform:
    def modify_parameters(parameters):
        TIME_PERIOD = "year:2022:10"
        rates = parameters.tax.income_tax.rates.uk.brackets
        basic_rate = rates[0].rate
        higher_rate = rates[1].rate
        additional_rate = rates[2].rate
        INSTANT = "2022-01-01"
        basic_rate.update(period=TIME_PERIOD, value=basic_rate(INSTANT) + basic_rate_addition)
        higher_rate.update(period=TIME_PERIOD, value=higher_rate(INSTANT) + higher_rate_addition)
        additional_rate.update(period=TIME_PERIOD, value=additional_rate(INSTANT) + additional_rate_addition)
        pa = parameters.tax.income_tax.allowances.personal_allowance.amount
        pa.update(period=TIME_PERIOD, value=personal_allowance)
        return parameters

    class reform(Reform):
        def apply(self):
            self.modify_parameters(modify_parameters)
    
    return reform


def get_revenue_slow_estimate(
    personal_allowance: float,
    basic_rate_addition: float,
    higher_rate_addition: float,
    additional_rate_addition: float,
) -> float:
    baseline, reformed = uk.create_microsimulations(dict(
        baseline_country_specific="NORTHERN_IRELAND",
        basic_rate=0.2 + basic_rate_addition,
        higher_rate=0.4 + higher_rate_addition,
        add_rate=0.45 + additional_rate_addition,
        personal_allowance=personal_allowance,
    ))
    return baseline.calc("household_net_income").sum() - reformed.calc("household_net_income").sum()

def get_revenue_quick_estimate(
    personal_allowance: float,
    basic_rate_addition: float,
    higher_rate_addition: float,
    additional_rate_addition: float,
) -> float:
    income = baseline.calc("adjusted_net_income")
    weight = income.weights
    add_rate_income = np.maximum(0, income - 150_000)
    higher_rate_income = np.maximum(0, income - 50_270) - add_rate_income
    basic_rate_income_b = np.maximum(0, income - higher_rate_income - add_rate_income - 12_570)
    basic_rate_income_r = np.maximum(0, income - higher_rate_income - add_rate_income - personal_allowance)
    basic_rate_increased_tax = (
        basic_rate_income_r * (0.2 + basic_rate_addition)
        - basic_rate_income_b * 0.2
    )
    higher_rate_increased_tax = baseline.calc("higher_rate_earned_income").values * higher_rate_addition
    additional_rate_increased_tax = baseline.calc("add_rate_earned_income").values * additional_rate_addition
    increased_tax = basic_rate_increased_tax + higher_rate_increased_tax + additional_rate_increased_tax
    return (increased_tax * weight).sum()

def solve_rates(
    adult_ubi: float = 0,
    personal_allowance: float = 12_570,
    higher_rate_ratio: float = 1,
    additional_rate_ratio: float = 1,
):
    def adjusted_reform_cost(
        basic_rate_addition: float = 0,
    ) -> float:
        """adjusted_reform = ubi_ni_reform(
            personal_allowance=personal_allowance,
            basic_rate_addition=basic_rate_addition,
            higher_rate_addition=higher_rate_ratio * basic_rate_addition,
            additional_rate_addition=additional_rate_ratio * basic_rate_addition,
        )"""
        adjusted_revenue = get_revenue_quick_estimate(personal_allowance, basic_rate_addition, basic_rate_addition * higher_rate_ratio, basic_rate_addition * additional_rate_ratio)
        ubi_cost = get_ubi_cost(adult_ubi, adult_ubi * 0.5, adult_ubi)
        return ubi_cost - adjusted_revenue
    
    return bisect(adjusted_reform_cost, -1, 1, xtol=1e-3)

In [3]:
import numpy as np
from tqdm import tqdm

headline_ubi_rates = []
personal_allowances = []
basic_rate_additions = []
higher_add_addition_ratios = []

for pa in tqdm([12_570] + list(range(12_000, -1000, -1000))):
    for ubi_rate in list([200 * 12, 300 * 12, 400 * 12]):
        for higher_add_ratio in list((0, 1, 2)):
            headline_ubi_rates.append(ubi_rate)
            personal_allowances.append(pa)
            basic_rate_additions.append(solve_rates(adult_ubi=ubi_rate, personal_allowance=pa, higher_rate_ratio=higher_add_ratio, additional_rate_ratio=higher_add_ratio))
            higher_add_addition_ratios.append(higher_add_ratio)

100%|██████████| 14/14 [00:20<00:00,  1.46s/it]


In [4]:
import plotly.express as px
from ubicenter import format_fig
from ubicenter.plotly import BLUE_COLOR_SEQUENCE
import pandas as pd

df = pd.DataFrame({
    "Adult UBI": headline_ubi_rates,
    "Personal Allowance": personal_allowances,
    "Basic rate addition": basic_rate_additions,
    "Higher/additional rate increase ratio": higher_add_addition_ratios,
})
df["Personal Allowance reduction"] = 12_570 - df["Personal Allowance"]
df["Higher rate addition"] = df["Basic rate addition"] * df["Higher/additional rate increase ratio"]
df["Basic rate addition (x100)"] = df["Basic rate addition"] * 100
df["Additional rate addition"] = df["Basic rate addition"] * df["Higher/additional rate increase ratio"]
df["UBI/Increase ratio combination"] = [
    f"{higher_add_ratio:.0%}" + " " * (ubi // 100)
    for ubi, higher_add_ratio in zip(
        df["Adult UBI"],
        df["Higher/additional rate increase ratio"],
    )
]
df["Label"] = [
    f"A £{ubi:,.0f} UBI and £{pa:,.0f} Personal Allowance can be made <br>budget neutral by setting the basic, higher and additional rates<br> to {0.2 + basic_rate_addition:.0%}, {0.4 + higher_rate_addition:.0%} and {0.45 + additional_rate_addition:.0%} respectively."
    for ubi, pa, basic_rate_addition, higher_rate_addition, additional_rate_addition in zip(
        df["Adult UBI"],
        df["Personal Allowance"],
        df["Basic rate addition"],
        df["Higher rate addition"],
        df["Additional rate addition"],
    )
]

df = df.sort_values(["Adult UBI", "Higher/additional rate increase ratio"])

df["Higher rate"] = df["Higher rate addition"] + 0.4
df["Add. rate"] = df["Additional rate addition"] + 0.45
df["PolicyEngine link"] = [
    f"https://policyengine.org/uk/population-impact?basic_rate={20 + basic_rate * 100:.1f}&higher_rate={40 + higher_rate * 100:.1f}&add_rate={45 + add_rate * 100:.1f}&personal_allowance={pa}&higher_threshold=50270&adult_bi={ubi/52:.2f}&senior_bi={ubi/52:.2f}&child_bi={ubi/104:.2f}&baseline_country_specific=NORTHERN_IRELAND"
    for basic_rate, higher_rate, add_rate, pa, ubi in zip(
        df["Basic rate addition"],
        df["Higher rate addition"],
        df["Additional rate addition"],
        df["Personal Allowance"],
        df["Adult UBI"],
    )
]
df.to_csv("experiment_results.csv")
df = df[
    (
        # £150k+ MTR under 100%
        (df["Add. rate"] + 0.02 < 1)
        # £100k-150k MTR under 100%
        & (df["Higher rate"] * (
            np.where(df["Personal Allowance"] > 0, 1.5, 1)
        ) + 0.02 < 1)
    )
]

fig = px.line(
    df, 
    x="Basic rate addition (x100)", 
    y="Personal Allowance reduction", 
    color="UBI/Increase ratio combination",  
    custom_data=[df.Label],
    color_discrete_sequence=BLUE_COLOR_SEQUENCE[:3],
)
fig.update_layout(
    xaxis_tickformat="+.0f",
    xaxis_title="Basic rate addition",
    xaxis_ticksuffix="p",
    xaxis_tickprefix="+",
    yaxis_tickprefix="£",
    yaxis_tickformat=",.0f",
    yaxis_range=(0, 12_570),
    xaxis_range=(0, 60),
    legend_title="Higher/add. rate increase ratio",
    title="Budget neutral UBI funding models",
)

CUSTOM_HOVERTEMPLATE="%{customdata[0]}"
fig.update_traces(hovertemplate=CUSTOM_HOVERTEMPLATE)

for i, data in enumerate(fig.data):
    data.hovertemplate = CUSTOM_HOVERTEMPLATE
    if i % 4 == 0:
        data["showlegend"] = True
    else:
        data["showlegend"] = False

import plotly.graph_objects as go

fig.add_trace(go.Scatter(
    x=np.linspace(15, 55, 3),
    y=[1_000] * 3,
    mode="text",
    name="Label",
    text=["£200/mo", "£300/mo", "£400/mo"],
    textposition="bottom center",
    showlegend=False,
))


format_fig(fig)

## Impacts of budget-neutral UBI plans

* Poverty
* Inequality
